# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,6.39,77,72,4.88,CL,1720091050
1,1,taksimo,56.3444,114.8794,23.68,68,94,1.88,RU,1720091460
2,2,port mathurin,-19.6833,63.4167,22.28,67,29,9.89,MU,1720090946
3,3,darzab,35.9774,65.3783,34.19,11,39,4.46,AF,1720091463
4,4,bodden town,19.2833,-81.2500,25.00,96,75,19.55,KY,1720091464


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot= city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City")    

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values  
# My ideal weather: Wind speed less than 5 m/s; Humidity less than 75
Ideal_cities = city_data_df.loc[(city_data_df["Wind Speed"]<5)&(city_data_df["Wind Speed"]>2)\
                                & (city_data_df["Humidity"]<50)&(city_data_df["Cloudiness"]<30) \
                                & (city_data_df["Max Temp"]<25) & (city_data_df["Max Temp"]>15)]

# Drop any rows with null values
Ideal_cities_clean = Ideal_cities.dropna()

# Display sample data
Ideal_cities_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,67,eldorado at santa fe,35.5264,-105.9347,17.20,45,0,2.08,US,1720091572
236,236,ghanzi,-21.5667,21.7833,24.04,15,0,3.70,BW,1720091864
239,239,capitan bado,-23.2667,-55.5333,18.25,48,0,3.48,PY,1720091869
297,297,namibe,-15.1961,12.1522,24.95,48,5,3.61,AO,1720091969
323,323,sao miguel do araguaia,-13.2750,-50.1628,22.62,49,0,2.08,BR,1720092012
356,356,kristiansund,63.1115,7.7320,20.36,42,0,4.63,NO,1720092067
457,457,rio verde de mato grosso,-18.9181,-54.8442,19.77,41,0,2.59,BR,1720092237


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_cities_clean[["City","Country","Lat","Lng","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

/var/folders/d7/tvjrqth10gb5pst1dd7n9wl40000gn/T/ipykernel_51825/1766342809.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
67,eldorado at santa fe,US,35.5264,-105.9347,45,
236,ghanzi,BW,-21.5667,21.7833,15,
239,capitan bado,PY,-23.2667,-55.5333,48,
297,namibe,AO,-15.1961,12.1522,48,
323,sao miguel do araguaia,BR,-13.2750,-50.1628,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 1

params = {"categories":categories,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url,params = params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
eldorado at santa fe - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
capitan bado - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pensao
sao miguel do araguaia - nearest hotel: Hotel São Miguel
kristiansund - nearest hotel: Comfort Hotel Fosna
rio verde de mato grosso - nearest hotel: Balneário e Pousada Paraíso


,City,Country,Lat,Lng,Humidity,Hotel Name
67,eldorado at santa fe,US,35.5264,-105.9347,45,No hotel found
236,ghanzi,BW,-21.5667,21.7833,15,Ghanzi farmhouse
239,capitan bado,PY,-23.2667,-55.5333,48,No hotel found
297,namibe,AO,-15.1961,12.1522,48,Nelsal Pensao
323,sao miguel do araguaia,BR,-13.2750,-50.1628,49,Hotel São Miguel
356,kristiansund,NO,63.1115,7.7320,42,Comfort Hotel Fosna
457,rio verde de mato grosso,BR,-18.9181,-54.8442,41,Balneário e Pousada Paraíso


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
map_plot_ideal= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Humidity",
    scale = 3,
    color = "City",
    hover_cols=["Hotel Name","City","Country","Lat","Lng","Humidity"])   

# Display the map
map_plot_ideal

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)